In [186]:
import pandas as pd
from pandas import json_normalize
import numpy as np
import json as js
import ast as ast
import re
import pyarrow as pa
import pyarrow.parquet as pq

## ETL 

### Carga de datos

In [2]:
games_org = pd.read_json('output_steam_games.json',lines=True)

In [3]:
games = games_org.copy()

In [4]:
list_rev = []
archivo1 = r'australian_user_reviews.json'
with open(archivo1, encoding='utf-8') as file:
    for line in file.readlines():
        list_rev.append(ast.literal_eval(line))

reviews_org = pd.DataFrame(list_rev)

In [5]:
reviews = reviews_org.copy()
reviews = reviews.explode('reviews',ignore_index=True)

In [6]:
list_items = []
archivo2 = r'australian_users_items.json'
with open(archivo2, encoding='utf-8') as file2:
    for line2 in file2.readlines():
        list_items.append(ast.literal_eval(line2))

items_org = pd.DataFrame(list_items)

In [212]:
items = items_org.copy()
items = items.explode('items',ignore_index=True)

### Revision de Games

#### Informacion general

In [8]:
games

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
1,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
2,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
3,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
4,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,0.0,773640.0,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,0.0,733530.0,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,0.0,658870.0,"xropi,stev3ns"


In [9]:
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24083 non-null  object 
 1   genres        28852 non-null  object 
 2   app_name      32133 non-null  object 
 3   title         30085 non-null  object 
 4   url           32135 non-null  object 
 5   release_date  30068 non-null  object 
 6   tags          31972 non-null  object 
 7   reviews_url   32133 non-null  object 
 8   specs         31465 non-null  object 
 9   price         30758 non-null  object 
 10  early_access  32135 non-null  float64
 11  id            32133 non-null  float64
 12  developer     28836 non-null  object 
dtypes: float64(2), object(11)
memory usage: 11.9+ MB


#### Normalizacion de columnas

In [10]:
games.drop(columns=['url','reviews_url','publisher','title'],inplace=True)

In [11]:
games.rename(columns={'id':'item_id','app_name':'item_name'},inplace=True)

In [12]:
games.head()

,genres,item_name,release_date,tags,specs,price,early_access,item_id,developer
0,None,None,None,None,None,None,NaN,NaN,None
1,None,None,None,None,None,None,NaN,NaN,None
2,None,None,None,None,None,None,NaN,NaN,None
3,None,None,None,None,None,None,NaN,NaN,None
4,None,None,None,None,None,None,NaN,NaN,None


#### Manejo de nulos

In [13]:
games.head()

,genres,item_name,release_date,tags,specs,price,early_access,item_id,developer
0,None,None,None,None,None,None,NaN,NaN,None
1,None,None,None,None,None,None,NaN,NaN,None
2,None,None,None,None,None,None,NaN,NaN,None
3,None,None,None,None,None,None,NaN,NaN,None
4,None,None,None,None,None,None,NaN,NaN,None


In [14]:
games.isnull().sum()

genres          91593
item_name       88312
release_date    90377
tags            88473
specs           88980
price           89687
early_access    88310
item_id         88312
developer       91609
dtype: int64

120444 registros de games antes de borrar vacios. Son 88309 registros vacios. 3135 registros

In [15]:
games.dropna(how='all',inplace=True)

Se revisaron los valores nulos de las columnas 'app_name' y 'item_id' ya que se consideraron las mas importantes. Se busco los id y nombres en el DataFrame items para completar la informacion, pero al no encontraste en este DataFrame se decidio borrar los registros. Se borraron 3 registros.

In [16]:
games[(games['item_name'].isnull())|(games['item_id'].isnull())]
#items[items['item_id']==317160.0]
#items[items['item_name']=='Batman: Arkham City - Game of the Year Edition']

,genres,item_name,release_date,tags,specs,price,early_access,item_id,developer
88384,None,None,None,None,None,19.99,0.0,NaN,None
90890,"[Action, Indie]",None,2014-08-26,"[Action, Indie]","[Single-player, Game demo]",None,0.0,317160.0,None
119271,"[Action, Adventure]",Batman: Arkham City - Game of the Year Edition,2012-09-07,"[Action, Open World, Batman, Adventure, Stealt...","[Single-player, Steam Achievements, Steam Trad...",19.99,0.0,NaN,"Rocksteady Studios,Feral Interactive (Mac)"


In [17]:
#Borra registro sin app_name y item_id
games.drop(games[(games['item_name'].isnull())|(games['item_id'].isnull())].index,inplace=True)

#### Manejo de duplicados

Se borro 1 duplicado

In [18]:
games.drop(games[games.duplicated(subset=['item_name', 'release_date', 'price', 'early_access', 'item_id',
       'developer'])].index,inplace=True)

#### Columna 'price'

Se revisaron los valores de la columna 'price' para poder normalizar y cambiar el tipo de dato

In [19]:
games['price'].unique()

array([4.99, 'Free To Play', 'Free to Play', 0.99, 2.99, 3.99, 9.99,
       18.99, 29.99, None, 'Free', 10.99, 1.5899999999999999, 14.99, 1.99,
       59.99, 8.99, 6.99, 7.99, 39.99, 19.99, 7.49, 12.99, 5.99, 2.49,
       15.99, 1.25, 24.99, 17.99, 61.99, 3.49, 11.99, 13.99, 'Free Demo',
       'Play for Free!', 34.99, 74.76, 1.49, 32.99, 99.99, 14.95, 69.99,
       16.99, 79.99, 49.99, 5.0, 44.99, 13.98, 29.96, 119.99, 109.99,
       149.99, 771.71, 'Install Now', 21.99, 89.99,
       'Play WARMACHINE: Tactics Demo', 0.98, 139.92, 4.29, 64.99,
       'Free Mod', 54.99, 74.99, 'Install Theme', 0.89, 'Third-party',
       0.5, 'Play Now', 299.99, 1.29, 3.0, 15.0, 5.49, 23.99, 49.0, 20.99,
       10.93, 1.3900000000000001, 'Free HITMAN™ Holiday Pack', 36.99,
       4.49, 2.0, 4.0, 9.0, 234.99, 1.9500000000000002, 1.5, 199.0, 189.0,
       6.66, 27.99, 10.49, 129.99, 179.0, 26.99, 399.99, 31.99, 399.0,
       20.0, 40.0, 3.33, 199.99, 22.99, 320.0, 38.85, 71.7, 59.95, 995.0,
       27.49,

Se cambiaron los valores de texto de la columna 'price' a 0.00 con excepcion de los valores 'Starting at $499.00' y 'Starting at $449.00' que se 
establecieron en los precios del texto ($499 y $449)

In [20]:
games['price']=games['price'].replace('Starting at $499.00',499.00)
games['price']=games['price'].replace('Starting at $449.00',449.00)

In [21]:
mask = games['price'].apply(lambda x: isinstance(x, str))
games.loc[mask, 'price'] = 0.00

In [22]:
games['price']=games['price'].astype(float)

In [78]:
games['price'].unique().size

148

#### Columna 'release_year'

In [23]:
#Funcion para extraer el año de la fecha de lanzamiento
def extract_year(value):
    
    match = re.search(r'\b(19|20)\d{2}\b', str(value))
    if match:
        return int(match.group(0)) 
    return None

In [24]:
#Columna solo con el año
games['release_year'] = games['release_date'].apply(extract_year)
games.drop(columns=['release_date'],inplace=True)

#### Version final

In [71]:
games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32131 entries, 88310 to 120444
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   genres        28849 non-null  object 
 1   item_name     32131 non-null  object 
 2   tags          31969 non-null  object 
 3   specs         31462 non-null  object 
 4   price         30755 non-null  float64
 5   early_access  32131 non-null  float64
 6   item_id       32131 non-null  float64
 7   developer     28834 non-null  object 
 8   release_year  29961 non-null  float64
dtypes: float64(4), object(5)
memory usage: 2.5+ MB


In [72]:
games.head()

,genres,item_name,tags,specs,price,early_access,item_id,developer,release_year
88310,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,"[Strategy, Action, Indie, Casual, Simulation]",[Single-player],4.99,0.0,761140.0,Kotoshiro,2018.0
88311,"[Free to Play, Indie, RPG, Strategy]",Ironbound,"[Free to Play, Strategy, Indie, RPG, Card Game...","[Single-player, Multi-player, Online Multi-Pla...",0.00,0.0,643980.0,Secret Level SRL,2018.0
88312,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,"[Free to Play, Simulation, Sports, Casual, Ind...","[Single-player, Multi-player, Online Multi-Pla...",0.00,0.0,670290.0,Poolians.com,2017.0
88313,"[Action, Adventure, Casual]",弹炸人2222,"[Action, Adventure, Casual]",[Single-player],0.99,0.0,767400.0,彼岸领域,2017.0
88314,None,Log Challenge,"[Action, Indie, Casual, Sports]","[Single-player, Full controller support, HTC V...",2.99,0.0,773570.0,None,NaN


### Revision de Reviews

#### Informacion general

In [25]:
reviews

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20..."
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted July 15, 2011...."
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted April 21, 2011..."
3,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted June 24, 2014...."
4,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted September 8, 2..."
...,...,...,...
59328,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"{'funny': '', 'posted': 'Posted July 10.', 'la..."
59329,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"{'funny': '', 'posted': 'Posted July 8.', 'las..."
59330,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,"{'funny': '1 person found this review funny', ..."
59331,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,"{'funny': '', 'posted': 'Posted July 20.', 'la..."


In [26]:
reviews.iloc[1,2]

{'funny': '',
 'posted': 'Posted July 15, 2011.',
 'last_edited': '',
 'item_id': '22200',
 'helpful': 'No ratings yet',
 'recommend': True,
 'review': "It's unique and worth a playthrough."}

#### Desanidado de la columna 'reviews'

In [27]:
reviews_expanded = pd.json_normalize(reviews['reviews'])
reviews_expanded=reviews_expanded.replace('',None)

In [28]:
reviews_expanded.head()

,funny,posted,last_edited,item_id,helpful,recommend,review
0,None,"Posted November 5, 2011.",None,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,None,"Posted July 15, 2011.",None,22200,No ratings yet,True,It's unique and worth a playthrough.
2,None,"Posted April 21, 2011.",None,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,None,"Posted June 24, 2014.",None,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,None,"Posted September 8, 2013.",None,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


In [29]:
reviews = reviews.join(reviews_expanded)
reviews.drop(columns=['reviews'],inplace=True)

In [30]:
reviews.head()

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,None,"Posted November 5, 2011.",None,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,None,"Posted July 15, 2011.",None,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,None,"Posted April 21, 2011.",None,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,http://steamcommunity.com/id/js41637,None,"Posted June 24, 2014.",None,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,http://steamcommunity.com/id/js41637,None,"Posted September 8, 2013.",None,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


#### Normalizacion de columnas

In [31]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59333 entries, 0 to 59332
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      59333 non-null  object
 1   user_url     59333 non-null  object
 2   funny        8151 non-null   object
 3   posted       59305 non-null  object
 4   last_edited  6140 non-null   object
 5   item_id      59305 non-null  object
 6   helpful      59305 non-null  object
 7   recommend    59305 non-null  object
 8   review       59275 non-null  object
dtypes: object(9)
memory usage: 4.1+ MB


In [32]:
reviews.drop(columns=['user_url','funny','posted','last_edited','helpful'],inplace=True)

In [33]:
reviews

,user_id,item_id,recommend,review
0,76561197970982479,1250,True,Simple yet with great replayability. In my opi...
1,76561197970982479,22200,True,It's unique and worth a playthrough.
2,76561197970982479,43110,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,251610,True,I know what you think when you see this title ...
4,js41637,227300,True,For a simple (it's actually not all that simpl...
...,...,...,...,...
59328,76561198312638244,70,True,a must have classic from steam definitely wort...
59329,76561198312638244,362890,True,this game is a perfect remake of the original ...
59330,LydiaMorley,273110,True,had so much fun plaing this and collecting res...
59331,LydiaMorley,730,True,:D


#### Manejo de nulos

In [34]:
reviews.isnull().sum()

user_id       0
item_id      28
recommend    28
review       58
dtype: int64

In [35]:
#28 registros que tienen datos nulos
reviews[reviews['item_id'].isnull()]

,user_id,item_id,recommend,review
137,gdxsd,NaN,NaN,NaN
177,76561198094224872,NaN,NaN,NaN
2559,76561198021575394,NaN,NaN,NaN
10080,cmuir37,NaN,NaN,NaN
13767,Jaysteeny,NaN,NaN,NaN
15493,ML8989,NaN,NaN,NaN
19184,76561198079215291,NaN,NaN,NaN
20223,76561198079342142,NaN,NaN,NaN
25056,76561198061996985,NaN,NaN,NaN
26257,76561198108286351,NaN,NaN,NaN


In [36]:
reviews.drop(reviews[reviews['item_id'].isnull()].index,inplace=True)

#### Manejo de duplicados

In [37]:
#Se borraron 874 duplicados
reviews.drop(reviews[reviews.duplicated()].index,inplace=True)

#### Cambio de tipo de datos - Columnas 'item_id' y 'recommend'

In [38]:
reviews['item_id']=reviews['item_id'].astype(float)

In [39]:
reviews['recommend']=reviews['recommend'].astype(bool)

#### Adicion de item_name al DataFrame

In [159]:
reviews = reviews.merge(games[['item_name','item_id']], on='item_id', how='left')

In [160]:
#lista de los item_id unicos en la tabla items
items_unicos = items.drop(items[items.duplicated(subset='item_id')].index)

In [161]:
items_unicos

,user_id,items_count,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,10.0,Counter-Strike,6.0,0.0
1,76561197970982479,277,20.0,Team Fortress Classic,0.0,0.0
2,76561197970982479,277,30.0,Day of Defeat,7.0,0.0
3,76561197970982479,277,40.0,Deathmatch Classic,0.0,0.0
4,76561197970982479,277,50.0,Half-Life: Opposing Force,0.0,0.0
...,...,...,...,...,...,...
4926140,76561198081168593,84,375450.0,NOBUNAGA'S AMBITION: Souzou SengokuRisshiden,51.0,0.0
5108879,76561198101994484,139,353390.0,Alienware Steam Machine,0.0,0.0
5121271,76561198107283457,33,354280.0,ChaosTower,164.0,0.0
5135856,inven,3603,433920.0,Aveyond 4: Shadow Of The Mist,0.0,0.0


In [162]:
reviews = reviews.merge(items_unicos[['item_id','item_name']], on='item_id', how='left')

In [163]:
reviews

,user_id,item_id,recommend,review,item_name_x,item_name_y
0,76561197970982479,1250.0,True,Simple yet with great replayability. In my opi...,Killing Floor,Killing Floor
1,76561197970982479,22200.0,True,It's unique and worth a playthrough.,Zeno Clash,Zeno Clash
2,76561197970982479,43110.0,True,Great atmosphere. The gunplay can be a bit chu...,NaN,Metro 2033
3,js41637,251610.0,True,I know what you think when you see this title ...,NaN,Barbie™ Dreamhouse Party™
4,js41637,227300.0,True,For a simple (it's actually not all that simpl...,Euro Truck Simulator 2,Euro Truck Simulator 2
...,...,...,...,...,...,...
58426,76561198312638244,70.0,True,a must have classic from steam definitely wort...,Half-Life,Half-Life
58427,76561198312638244,362890.0,True,this game is a perfect remake of the original ...,Black Mesa,Black Mesa
58428,LydiaMorley,273110.0,True,had so much fun plaing this and collecting res...,Counter-Strike Nexon: Zombies,Counter-Strike Nexon: Zombies
58429,LydiaMorley,730.0,True,:D,Counter-Strike: Global Offensive,Counter-Strike: Global Offensive


In [164]:
reviews['item_name'] = reviews['item_name_x'].combine_first(reviews['item_name_y'])

In [165]:
reviews.drop(columns=['item_name_x','item_name_y'],inplace=True)

In [166]:
reviews

,user_id,item_id,recommend,review,item_name
0,76561197970982479,1250.0,True,Simple yet with great replayability. In my opi...,Killing Floor
1,76561197970982479,22200.0,True,It's unique and worth a playthrough.,Zeno Clash
2,76561197970982479,43110.0,True,Great atmosphere. The gunplay can be a bit chu...,Metro 2033
3,js41637,251610.0,True,I know what you think when you see this title ...,Barbie™ Dreamhouse Party™
4,js41637,227300.0,True,For a simple (it's actually not all that simpl...,Euro Truck Simulator 2
...,...,...,...,...,...
58426,76561198312638244,70.0,True,a must have classic from steam definitely wort...,Half-Life
58427,76561198312638244,362890.0,True,this game is a perfect remake of the original ...,Black Mesa
58428,LydiaMorley,273110.0,True,had so much fun plaing this and collecting res...,Counter-Strike Nexon: Zombies
58429,LydiaMorley,730.0,True,:D,Counter-Strike: Global Offensive


#### Version final

In [167]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58431 entries, 0 to 58430
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   user_id    58431 non-null  object 
 1   item_id    58431 non-null  float64
 2   recommend  58431 non-null  bool   
 3   review     58401 non-null  object 
 4   item_name  58227 non-null  object 
dtypes: bool(1), float64(1), object(3)
memory usage: 1.8+ MB


In [168]:
reviews.isnull().sum()

user_id        0
item_id        0
recommend      0
review        30
item_name    204
dtype: int64

In [169]:
reviews

,user_id,item_id,recommend,review,item_name
0,76561197970982479,1250.0,True,Simple yet with great replayability. In my opi...,Killing Floor
1,76561197970982479,22200.0,True,It's unique and worth a playthrough.,Zeno Clash
2,76561197970982479,43110.0,True,Great atmosphere. The gunplay can be a bit chu...,Metro 2033
3,js41637,251610.0,True,I know what you think when you see this title ...,Barbie™ Dreamhouse Party™
4,js41637,227300.0,True,For a simple (it's actually not all that simpl...,Euro Truck Simulator 2
...,...,...,...,...,...
58426,76561198312638244,70.0,True,a must have classic from steam definitely wort...,Half-Life
58427,76561198312638244,362890.0,True,this game is a perfect remake of the original ...,Black Mesa
58428,LydiaMorley,273110.0,True,had so much fun plaing this and collecting res...,Counter-Strike Nexon: Zombies
58429,LydiaMorley,730.0,True,:D,Counter-Strike: Global Offensive


### Revision de Items

#### Informacion general

In [213]:
items

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '10', 'item_name': 'Counter-Strike..."
1,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '20', 'item_name': 'Team Fortress ..."
2,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '30', 'item_name': 'Day of Defeat'..."
3,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '40', 'item_name': 'Deathmatch Cla..."
4,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '50', 'item_name': 'Half-Life: Opp..."
...,...,...,...,...,...
5170010,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"{'item_id': '373330', 'item_name': 'All Is Dus..."
5170011,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"{'item_id': '388490', 'item_name': 'One Way To..."
5170012,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"{'item_id': '521570', 'item_name': 'You Have 1..."
5170013,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"{'item_id': '519140', 'item_name': 'Minds Eyes..."


In [214]:
items.iloc[0,4]

{'item_id': '10',
 'item_name': 'Counter-Strike',
 'playtime_forever': 6,
 'playtime_2weeks': 0}

#### Desanidado de la columna 'items'

In [215]:
items_expanded = pd.json_normalize(items['items'])
items_expanded=items_expanded.replace('',None)

In [216]:
items_expanded.head()

,item_id,item_name,playtime_forever,playtime_2weeks
0,10,Counter-Strike,6.0,0.0
1,20,Team Fortress Classic,0.0,0.0
2,30,Day of Defeat,7.0,0.0
3,40,Deathmatch Classic,0.0,0.0
4,50,Half-Life: Opposing Force,0.0,0.0


In [217]:
items = items.join(items_expanded)
items.drop(columns=['items'],inplace=True)

In [218]:
items.head()

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6.0,0.0
1,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,20,Team Fortress Classic,0.0,0.0
2,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,30,Day of Defeat,7.0,0.0
3,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,40,Deathmatch Classic,0.0,0.0
4,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,50,Half-Life: Opposing Force,0.0,0.0


#### Normalizacion de columnas

In [219]:
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5170015 entries, 0 to 5170014
Data columns (total 8 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   items_count       int64  
 2   steam_id          object 
 3   user_url          object 
 4   item_id           object 
 5   item_name         object 
 6   playtime_forever  float64
 7   playtime_2weeks   float64
dtypes: float64(2), int64(1), object(5)
memory usage: 315.6+ MB


- __'items_count'__ contiene informacion repetida que se puede obtener a traves de la funcion values_count()
- __'playtime_2weeks'__ corresponde al tiempo de juego del usuario en las ultimas 2 semanas. Teniendo en cuenta que para el objetivo del analisis es necesario conocer el tiempo total de juego del usuario y que no se conoce la fecha exacta de la extraccion de los datos se considero que este no aporta informacion de valor para el analisis. De acuerdo con esto, se decidio eliminar la columna. 

In [220]:
items.drop(columns=['user_url','steam_id','items_count','playtime_2weeks'],inplace=True)

In [233]:
items.rename(columns={'playtime_forever':'playtime'},inplace=True)

In [234]:
items

,user_id,item_id,item_name,playtime
0,76561197970982479,10.0,Counter-Strike,0.100000
1,76561197970982479,20.0,Team Fortress Classic,0.000000
2,76561197970982479,30.0,Day of Defeat,0.116667
3,76561197970982479,40.0,Deathmatch Classic,0.000000
4,76561197970982479,50.0,Half-Life: Opposing Force,0.000000
...,...,...,...,...
5170009,76561198329548331,346330.0,BrainBread 2,0.000000
5170010,76561198329548331,373330.0,All Is Dust,0.000000
5170011,76561198329548331,388490.0,One Way To Die: Steam Edition,0.050000
5170012,76561198329548331,521570.0,You Have 10 Seconds 2,0.066667


#### Manejo de nulos

In [235]:
items.isnull().sum()

user_id      0
item_id      0
item_name    0
playtime     0
dtype: int64

In [236]:
#Se borraron 16.806 nulos
items.dropna(inplace=True)

#### Manejo de duplicados

In [237]:
#Se borran 59.104 duplicados
items.drop(items[items.duplicated()].index,inplace=True)

#### Cambio tipo de datos - Columna 'item_id'

In [238]:
items['item_id']=items['item_id'].astype(float)

#### Conversion de tiempos - Columna 'playtime_forever'

Se convierten los tiempos de minutos a horas para facilitar el analisis y que se encuentre en las unidades correspondientes a las consultas que se realizaran mas adelante.

In [239]:
items['playtime']=items['playtime']/60

#### Version final

In [240]:
items.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5094092 entries, 0 to 5170013
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   user_id    object 
 1   item_id    float64
 2   item_name  object 
 3   playtime   float64
dtypes: float64(2), object(2)
memory usage: 194.3+ MB


In [241]:
items.isnull().sum()

user_id      0
item_id      0
item_name    0
playtime     0
dtype: int64

In [242]:
items.head()

,user_id,item_id,item_name,playtime
0,76561197970982479,10.0,Counter-Strike,0.001667
1,76561197970982479,20.0,Team Fortress Classic,0.000000
2,76561197970982479,30.0,Day of Defeat,0.001944
3,76561197970982479,40.0,Deathmatch Classic,0.000000
4,76561197970982479,50.0,Half-Life: Opposing Force,0.000000


### Exportar DataFrames finales

In [187]:
pq.write_table(pa.Table.from_pandas(games),'games.parquet')

In [188]:
pq.write_table(pa.Table.from_pandas(reviews),'reviews.parquet')

In [244]:
pq.write_table(pa.Table.from_pandas(items),'items.parquet')